# Head and Shoulders

In [10]:
import pandas as pd
import os
df = pd.read_csv(os.getcwd()+"/backtest/BTCUSDT.csv")
df =   df.drop(df.columns[0], axis=1)
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,time,open,high,low,close,volume
0,1502942400000,4261.48,4349.99,4261.32,4349.99,82.088865
1,1502956800000,4333.32,4485.39,4333.32,4427.30,63.619882
2,1502971200000,4436.06,4485.39,4333.42,4352.34,174.562001
3,1502985600000,4352.33,4354.84,4200.74,4325.23,225.109716
4,1503000000000,4307.56,4369.69,4258.56,4285.08,249.769913
5,1503014400000,4285.08,4340.62,4134.61,4292.39,276.193043
6,1503028800000,4292.39,4340.62,4234.43,4300.25,248.389029
7,1503043200000,4285.00,4371.52,4259.85,4340.31,196.140129
8,1503057600000,4320.52,4340.31,4193.70,4236.89,175.969384
9,1503072000000,4234.54,4297.75,4015.70,4136.28,193.646970


In [11]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,15,15), axis=1)
df['shortpivot'] = df.apply(lambda x: pivotid(df, x.name,5,5), axis=1)

In [12]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def shortpointpos(x):
    if x['shortpivot']==1:
        return x['low']-2e-3
    elif x['shortpivot']==2:
        return x['high']+2e-3
    else:
        return np.nan
    
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['shortpointpos'] = df.apply(lambda row: shortpointpos(row), axis=1)

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[6750:6880]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.add_scatter(x=dfpl.index, y=dfpl['shortpointpos'], mode="markers",
                marker=dict(size=5, color="red"),
                name="shortpivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [32]:
from scipy.stats import linregress
backcandles= 14 #!!!should be less than pivot candles

for candleid in range(0, len(df)-backcandles):
    if df.iloc[candleid].pivot != 2 or df.iloc[candleid].shortpivot != 2:
        continue
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    minbcount=0 #minimas before head
    maxbcount=0 #maximas before head
    minacount=0 #minimas after head
    maxacount=0 #maximas after head
    
    for i in range(candleid-backcandles, candleid+backcandles):
        if df.iloc[i].shortpivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name             
            if i < candleid:
                minbcount=+1
            elif i>candleid:
                minacount+=1
        if df.iloc[i].shortpivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
            if i < candleid:
                maxbcount+=1
            elif i>candleid:
                maxacount+=1
    if minbcount<1 or minacount<1 or maxbcount<1 or maxacount<1:
        continue

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    headidx = np.argmax(maxim, axis=0)
    if maxim[headidx]-maxim[headidx-1]>30 and maxim[headidx]-maxim[headidx+1]>30 and abs(slmin)<=10 and xxmin[0]>xxmax[headidx-1] and xxmin[1]<xxmax[headidx+1]:# and (maxim[headidx]-maxim[headidx+1])>(maxim[headidx+1]-minim[headidx+1]) and (maxim[headidx]-maxim[headidx-1])>(maxim[headidx-1]-minim[headidx-1]) :
        print(minbcount,minacount,maxbcount,maxacount, slmin, candleid)
        #print(maxim)
        #print(xxmax)
        #print(minim)
        #print(xxmin)
        
    if candleid % 1000 == 0:
        print(candleid)

1 1 1 1 -4.936666666666649 277
1 1 1 1 -1.3424999999999727 1917
1 1 1 1 3.8975000000000364 6810
1 1 1 1 -6.158333333333252 8300
